In [8]:
import os
import random
import pickle
from pathlib import Path
from collections import defaultdict

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

SEED = 42
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

from scipy.signal import butter, filtfilt

In [13]:
def lowpass_filter(data, cutoff=2.0, fs=20.0, order=2):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return filtfilt(b, a, data)


def fit_scalers(file_paths, sample_size=5000):
    sample_paths = random.sample(file_paths, min(len(file_paths), sample_size))
    dfs = [pd.read_csv(p)[SAVE_COLUMNS].rename(columns=RENAME_COLUMNS) for p in sample_paths]
    # dfs = process_map(pd.read_csv, files, max_workers=24, chunksize=100)
    df = pd.concat(dfs) # , ignore_index=True)

    scalers = {
        'roll': StandardScaler().fit(df[['roll']].values),
        'aEgo': StandardScaler().fit(df[['aEgo']].values),
        'vEgo': StandardScaler().fit(df[['vEgo']].values),
        'targetLateralAcceleration': StandardScaler().fit(df[['targetLateralAcceleration']].values), # RobustScaler() 
        # 'steerCommand': RobustScaler().fit(df[['steerCommand']].values)
    }
    return scalers


class DrivingDataset(Dataset):
    def __init__(self, file_paths, seq_len=SEQ_LEN, future_k=FUTURE_K, scalers=None):
        self.file_paths = file_paths
        self.seq_len = seq_len
        self.future_k = future_k
        self.scalers = scalers

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        path = self.file_paths[idx]
        df = pd.read_csv(path)
        df = df[SAVE_COLUMNS].rename(columns=RENAME_COLUMNS)

        # Need at least seq_len + future horizon + 1 rows
        min_needed = self.seq_len + self.future_k
        if len(df) < min_needed:
            return self[random.randint(0, len(self) - 1)]  # Skip short episodes

        # Apply standard scalers
        for col, scaler in self.scalers.items():
            df[col] = scaler.transform(df[[col]].values)

        # Per-file robust scaling for steerCommand using first 100 rows (or entire if shorter)
        first_100 = df.iloc[:min(100, len(df))]
        steer_scaler = RobustScaler().fit(first_100[['steerCommand']].values)
        df['steerCommand'] = steer_scaler.transform(df[['steerCommand']].values)
        # df['steerCommand'] = lowpass_filter(df['steerCommand'].values)
        # df['steerCommand'] = df['steerCommand'].rolling(5, min_periods=1, center=True).mean()

        
        # df['steerCommand'] = self.scalers['steerCommand'].transform(df[['steerCommand']].values)


        arr = df[['roll', 'aEgo', 'vEgo', 'targetLateralAcceleration', 'steerCommand']].values
        physics_input = arr[:self.seq_len, :3]
        control_input = arr[:self.seq_len, 3:5]  # targetLatAcc, steerCommand
        
        # Future plan: first FUTURE_K future target lat acc values immediately after first row
        fut_raw = arr[1:1 + self.future_k, 3]  # column index 3 = targetLateralAcceleration
        # Pad if needed (should not usually happen due to min_needed) but be safe
        if len(fut_raw) < self.future_k:
            pad_val = fut_raw[-1] if len(fut_raw) > 0 else 0.0
            fut_raw = np.pad(fut_raw, (0, self.future_k - len(fut_raw)), constant_values=pad_val)

        # Repeat future plan for each timestep and concatenate to control inputs
        future_plan_matrix = np.repeat(fut_raw.reshape(1, -1), self.seq_len, axis=0)  # (seq_len, FUTURE_K)
        control_aug = np.concatenate([control_input, future_plan_matrix], axis=1)      # (seq_len, 2 + FUTURE_K)

        # Labels: predict next-step steerCommand for each timestep
        y = arr[1:self.seq_len + 1, 4].reshape(-1, 1)

        return (
            torch.tensor(physics_input, dtype=torch.float32),
            torch.tensor(control_aug, dtype=torch.float32),
            torch.tensor(y, dtype=torch.float32)
        )


class LstmEncoderDecoder(nn.Module):
    def __init__(self, physics_input_size, control_input_size, hidden_size, num_layers, dropout=0.2):
        super().__init__()
        self.physics_encoder = nn.LSTM(physics_input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.control_encoder = nn.LSTM(control_input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.decoder = nn.LSTM(control_input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc_out = nn.Linear(hidden_size, 1)

    def forward(self, input_physics, input_control_sequence):
        _, (hidden_phsc, cell_phsc) = self.physics_encoder(input_physics)
        _, (hidden_ctrl, cell_ctrl) = self.control_encoder(input_control_sequence)

        hidden = (hidden_phsc + hidden_ctrl) / 2
        cell = (cell_phsc + cell_ctrl) / 2

        decoder_output, _ = self.decoder(input_control_sequence, (hidden, cell))
        return self.fc_out(decoder_output)


def train_val_split(file_paths, val_ratio=0.2, seed=SEED):
    random.seed(seed)
    shuffled = list(file_paths)
    random.shuffle(shuffled)
    split_idx = int(len(shuffled) * (1 - val_ratio))
    return shuffled[:split_idx], shuffled[split_idx:]


def train_model(model, model_save_path, train_loader, val_loader, num_epochs=NUM_EPOCHS, lr=LR):
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    best_val_loss = float('inf')
    history = defaultdict(list)
    history = {
        'epoch': [],
        'train_loss': [], 'train_mae': [], 'train_rmse': [], 'train_r2': [],
        'val_loss': [], 'val_mae': [], 'val_rmse': [], 'val_r2': []
    }

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_preds_all, train_tgts_all = [], []

        for physics, control, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Train"):
            physics, control, y = physics.to(DEVICE), control.to(DEVICE), y.to(DEVICE)

            optimizer.zero_grad()
            pred = model(physics, control)
            loss = criterion(pred, y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            train_loss += loss.item()

            train_preds_all.append(pred.detach().cpu().numpy())
            train_tgts_all.append(y.detach().cpu().numpy())

        avg_train_loss = train_loss / len(train_loader)
        train_preds = np.vstack(train_preds_all).reshape(-1)  # Flatten to 1D
        train_targets = np.vstack(train_tgts_all).reshape(-1)
        train_mae = mean_absolute_error(train_targets, train_preds)
        train_rmse = mean_squared_error(train_targets, train_preds, squared=False)
        train_r2 = r2_score(train_targets, train_preds)

        model.eval()
        val_loss = 0.0
        val_preds_all, val_tgts_all = [], []

        with torch.no_grad():
            for physics, control, y in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Val"):
                physics, control, y = physics.to(DEVICE), control.to(DEVICE), y.to(DEVICE)
                pred = model(physics, control)
                loss = criterion(pred, y)
                val_loss += loss.item()

                val_preds_all.append(pred.detach().cpu().numpy())
                val_tgts_all.append(y.detach().cpu().numpy())

        avg_val_loss = val_loss / len(val_loader)
        val_preds = np.vstack(val_preds_all).reshape(-1)  # Flatten to 1D
        val_targets = np.vstack(val_tgts_all).reshape(-1)
        val_mae = mean_absolute_error(val_targets, val_preds)
        val_rmse = mean_squared_error(val_targets, val_preds, squared=False)
        val_r2 = r2_score(val_targets, val_preds)

        print(
            f"Epoch {epoch+1}/{num_epochs} | "
            f"Train Loss: {avg_train_loss:.4f}, MAE: {train_mae:.4f}, RMSE: {train_rmse:.4f}, R2: {train_r2:.4f} | "
            f"Val Loss: {avg_val_loss:.4f}, MAE: {val_mae:.4f}, RMSE: {val_rmse:.4f}, R2: {val_r2:.4f}"
        )
        
        # Save history
        history['epoch'].append(epoch + 1)
        history['train_loss'].append(avg_train_loss)
        history['train_mae'].append(train_mae)
        history['train_rmse'].append(train_rmse)
        history['train_r2'].append(train_r2)
        history['val_loss'].append(avg_val_loss)
        history['val_mae'].append(val_mae)
        history['val_rmse'].append(val_rmse)
        history['val_r2'].append(val_r2)

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), model_save_path / 'lstm_future_best.pt')

    torch.save(model.state_dict(), model_save_path / 'lstm_future_final.pt')            

    # Save history to CSV
    history_df = pd.DataFrame(history)
    history_df.to_csv(f"{model_save_path}/history.csv", index=False)

    with open(model_save_path / 'history.pkl', 'wb') as f:
        pickle.dump(dict(history), f)

    return model

In [14]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA_DIR = Path("../data/train")

LSTM_HIDDEN_SIZE = 256
LSTM_NUM_LAYERS = 2
SEQ_LEN = 100
FUTURE_K = 20        # number of future target lataccel points (Option A)
BATCH_SIZE = 64
NUM_EPOCHS = 20
NUM_WORKERS = 4
LR = 1e-3


SCALER_TYPE = "local-scaler"
# SCALER_TYPE = "global-scaler"
TARGET_COLUMN = "steer-filtered"

NUMBER_SCENES_IN_DATASET = 10000

MODEL_NUMBER_VERSION = 4
MODEL_VERSION = f"v{MODEL_NUMBER_VERSION}_lstm-{LSTM_NUM_LAYERS}-{LSTM_HIDDEN_SIZE}_{NUMBER_SCENES_IN_DATASET}-dataset_lr-{LR}_epochs-{NUM_EPOCHS}_seq-{SEQ_LEN}_{SCALER_TYPE}_futurek-{FUTURE_K}_target-{TARGET_COLUMN}"
MODEL_SAVE_PATH = Path(f"../models/{MODEL_VERSION}")
MODEL_SAVE_PATH.mkdir(parents=True, exist_ok=True)

SAVE_COLUMNS = ['roll', 'aEgo', 'vEgo', 'latAccelSteeringAngle', 'steerFiltered']
RENAME_COLUMNS = {
    'latAccelSteeringAngle': 'targetLateralAcceleration',
    'steerFiltered': 'steerCommand'
}


In [ ]:
    all_files = list(DATA_DIR.glob("**/*.csv"))
    all_files = all_files[:NUMBER_SCENES_IN_DATASET]
    # print(len(all_files))
    train_files, val_files = train_val_split(all_files, val_ratio=0.2)
    
    scalers = fit_scalers(train_files)
    
    for s in scalers.values():
        if hasattr(s, 'feature_names_in_'):
            s.feature_names_in_ = None
    with open(f"{MODEL_SAVE_PATH}/scalers.pkl", "wb") as f:
        pickle.dump(scalers, f)
    
    train_dataset = DrivingDataset(train_files, seq_len=SEQ_LEN, scalers=scalers)
    val_dataset = DrivingDataset(val_files, seq_len=SEQ_LEN, scalers=scalers)
    
    # print(f"Number of validation files: {len(val_files)}")
    # print(f"Number of samples in validation dataset: {len(val_dataset)}")
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                              num_workers=NUM_WORKERS, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False,
                            num_workers=NUM_WORKERS, pin_memory=True)
    
    
    
    model = LstmEncoderDecoder(
        physics_input_size=3,
        control_input_size=2,
        hidden_size=LSTM_HIDDEN_SIZE,
        num_layers=LSTM_NUM_LAYERS
    ).to(DEVICE)
    
    
    model = train_model(model, MODEL_SAVE_PATH, train_loader, val_loader,
                        num_epochs=NUM_EPOCHS, lr=1e-3)
    
    # torch.save(model.state_dict(), f"{MODEL_SAVE_PATH}/lstm_final.pt")

Epoch 1/20 - Val: 100%|██████████| 32/32 [00:03<00:00, 10.31it/s]


Epoch 1/20 | Train Loss: 379471.5922, MAE: 4.1199, RMSE: 616.0125, R2: 0.0001 | Val Loss: 72.6342, MAE: 0.4389, RMSE: 8.6238, R2: 0.0560


Epoch 2/20 - Val: 100%|██████████| 32/32 [00:03<00:00, 10.32it/s]


Epoch 2/20 | Train Loss: 379374.3573, MAE: 4.0810, RMSE: 615.9336, R2: 0.0003 | Val Loss: 71.0176, MAE: 0.4347, RMSE: 8.5273, R2: 0.0770


Epoch 3/20 - Val: 100%|██████████| 32/32 [00:03<00:00, 10.25it/s]


...

Epoch 13/20 - Val: 100%|██████████| 32/32 [00:03<00:00, 10.38it/s]


Epoch 13/20 | Train Loss: 378997.9311, MAE: 4.0691, RMSE: 615.6280, R2: 0.0013 | Val Loss: 60.4428, MAE: 0.4208, RMSE: 7.8668, R2: 0.2145


Epoch 14/20 - Val: 100%|██████████| 32/32 [00:03<00:00,  9.86it/s]


Epoch 14/20 | Train Loss: 378955.7464, MAE: 4.0669, RMSE: 615.5939, R2: 0.0014 | Val Loss: 60.0339, MAE: 0.4259, RMSE: 7.8401, R2: 0.2198


Epoch 15/20 - Train:  18%|█▊        | 22/125 [00:02<00:10, 10.22it/s]

In [ ]:
        
    # # Plot loss and MAE
    # plt.figure(figsize=(12, 6))
    # plt.subplot(1, 2, 1)
    # plt.plot(history['epoch'], history['train_loss'], label='Train Loss')
    # plt.plot(history['epoch'], history['val_loss'], label='Val Loss')
    # plt.title('Loss Curve')
    # plt.xlabel('Epoch')
    # plt.ylabel('MSE Loss')
    # plt.grid()
    # plt.legend()

    # plt.subplot(1, 2, 2)
    # plt.plot(history['epoch'], history['train_mae'], label='Train MAE')
    # plt.plot(history['epoch'], history['val_mae'], label='Val MAE')
    # plt.title('MAE Curve')
    # plt.xlabel('Epoch')
    # plt.ylabel('MAE')
    # plt.grid()
    # plt.legend()

    # plt.tight_layout()
    # plt.savefig(f"{model_save_path}/loss_and_mae_plot.png")
    # plt.close()